In [1]:
import csv
import numpy as np

In [2]:
def read_file(fname):
    labels = {'exclusive': 0, 'item': 1, 'sku': 2}
    content = []
    with open(fname, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=';')
        for nn, row in enumerate(spamreader):
            if len(row) == 3 and nn > 0:
                content.append(({"preoffer": row[0], "item": row[1]}, labels[row[2]]))
    return content

In [3]:
trainset = read_file("/Users/danilonunes/workspace/dataset/trainset.csv")
testset = read_file("/Users/danilonunes/workspace/dataset/testset.csv")
validset = read_file("/Users/danilonunes/workspace/dataset/validationset.csv")

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence 

Using TensorFlow backend.


In [5]:
tokenizer = Tokenizer(nb_words=20000)

/Users/danilonunes/.virtualenvs/imret2/lib/python2.7/site-packages/keras/preprocessing/text.py:89: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [6]:
tokenizer.fit_on_texts([" ".join([text["preoffer"], text["item"]]) for text, _ in trainset])

In [7]:
def pad_sequences(tokenizer, sequences, maxlen=5000):
    preoffer = sequence.pad_sequences(tokenizer.sequences_to_matrix([line["preoffer"] for line, _ in sequences]), maxlen=maxlen)
    item = sequence.pad_sequences(tokenizer.sequences_to_matrix([line["item"] for line, _ in sequences]), maxlen=maxlen)
    labels = np.zeros((len(item), 3), dtype=np.int32)
    for idx, (_, label) in enumerate(sequences):
        labels[idx, label] = 1.
    return preoffer, item, labels

In [8]:
X_train_preoffer, X_train_item, y_train = pad_sequences(tokenizer, trainset)

In [9]:
X_train_preoffer.shape, X_train_item.shape, y_train.shape

((11282, 5000), (11282, 5000), (11282, 3))

In [10]:
X_val_preoffer, X_val_item, y_val = pad_sequences(tokenizer, validset)
X_test_preoffer, X_test_item, y_test = pad_sequences(tokenizer, testset)

In [11]:
from keras.layers import Input, Dense, Embedding, merge, Convolution2D, MaxPooling2D, Dropout, concatenate, Merge
from sklearn.cross_validation import train_test_split
from keras.layers.core import Reshape, Flatten
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model

/Users/danilonunes/.virtualenvs/imret2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
sequence_length = 5000
vocabulary_size = len(tokenizer.word_index) + 1
embedding_dim = 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

In [43]:
# this returns a tensor
preoffer = Input(shape=(sequence_length, ), dtype='int32')
item = Input(shape=(sequence_length, ), dtype='int32')
inputs = concatenate([preoffer, item])

embedding = Embedding(output_dim=embedding_dim, input_dim=vocabulary_size, input_length=2 * sequence_length)(inputs)
reshape = Reshape((2 * sequence_length, embedding_dim,1))(embedding)

conv_0 = Convolution2D(num_filters, filter_sizes[0], embedding_dim, border_mode='valid', init='normal', activation='relu', dim_ordering='tf')(reshape)
conv_1 = Convolution2D(num_filters, filter_sizes[1], embedding_dim, border_mode='valid', init='normal', activation='relu', dim_ordering='tf')(reshape)
conv_2 = Convolution2D(num_filters, filter_sizes[2], embedding_dim, border_mode='valid', init='normal', activation='relu', dim_ordering='tf')(reshape)

maxpool_0 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), border_mode='valid', dim_ordering='tf')(conv_0)
maxpool_1 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), border_mode='valid', dim_ordering='tf')(conv_1)
maxpool_2 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), border_mode='valid', dim_ordering='tf')(conv_2)

merged_tensor = merge([maxpool_0, maxpool_1, maxpool_2], mode='concat', concat_axis=1)
flatten = Flatten()(merged_tensor)
# reshape = Reshape((3*num_filters,))(merged_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(output_dim=3, activation='softmax')(dropout)

# this creates a model that includes
model = Model(input=[preoffer, item], output=output)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 256), kernel_initializer="normal", activation="relu", data_format="channels_last", padding="valid")`
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (4, 256), kernel_initializer="normal", activation="relu", data_format="channels_last", padding="valid")`
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (5, 256), kernel_initializer="normal", activation="relu", data_format="channels_last", padding="valid")`
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:13: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(padding="valid", strides=(1, 1), data_format="channels_last", pool_size=(4998, 1))`
/usr/local/lib/python2.7/site-packages/ipykerne

In [44]:
checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
model.compile(optimizer="rmsprop", loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
train_batch = gen_batch(X_train_preoffer, X_train_item, y_train)
valid_batch = gen_batch(X_val_preoffer, X_val_item, y_val, batch_size=4)
test_batch = gen_batch(X_test_preoffer, X_test_item, y_test)

In [ ]:
model.fit([X_train_preoffer, X_train_item], 
          y_train, 
          batch_size=32, epochs=1, verbose=1, 
          callbacks=[checkpoint], 
          validation_data=([X_val_preoffer, X_val_item], y_val))  # starts training


Train on 11282 samples, validate on 2423 samples
Epoch 1/1


In [ ]:
model.save("weights.hdf5")

In [ ]:
model.evaluate([X_test_preoffer, X_test_item], y_test, batch_size=32)